# Predicting Earnings Surprises

## Task

We want to predict the magnitude of company's upcoming earnings announcement using a machine learning classification model. The model is trained on three types of data: earnings, pricing, and technical price action data. The optimized model outputs a result into one of three classes: positive, neutral, or negative. A 'positive' classification indicates a predicted surprise >15% of the estimated eps, a 'negative' classification indicates a predicted surprise <-15% of the estimated eps, and a 'neutral' classification indicates no predicted surprise (15% < x < -15%). 

## Data

The data for training and testing the model came from several external data providers. Earning and pricing data was collected from Financial Modeling Prep's historical earnings calendar and daily indicator endpoints. Technical data is collected from FMP Cloud's daily technical indicator endpoint.

The schema below outlines the database architecture into an AWS RDS MySQL database:

![Untitled Workspace (1)](https://user-images.githubusercontent.com/45079557/150410944-eb8c8e30-ac2d-4f23-bb03-cb5c3f489cfb.png)

In [1]:
import pandas as pd
import numpy as np
import pymysql
from decouple import config
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
# Set parameters for AWS database
aws_hostname = config("AWS_HOST")
aws_database = config("AWS_DB")
aws_username = config("AWS_USER")
aws_password = config("AWS_PASS")
aws_port = config("AWS_PORT")

# Pull API keys from .env file
FMP_API_KEY = config("FMP_API_KEY")
FMP_CLOUD_API_KEY = config("FMP_CLOUD_API_KEY")

In [3]:
db = pymysql.connect(host=aws_hostname,user=aws_username, password=aws_password, database='rds-python', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()

### Retrieve Data from MySQL Database

In [4]:
cursor.execute("SELECT e.*, p.*, t.* FROM earnings e INNER JOIN pricing p ON e.id = p.id INNER JOIN technicals t ON e.id = t.id")
train = cursor.fetchall()

In [10]:
train_df = pd.DataFrame(train)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75781 entries, 0 to 75780
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 75781 non-null  object 
 1   earnings_date      75781 non-null  object 
 2   symbol             75781 non-null  object 
 3   eps                75578 non-null  float64
 4   eps_estimated      75781 non-null  float64
 5   earnings_time      75781 non-null  object 
 6   p.id               75781 non-null  object 
 7   p.earnings_date    75781 non-null  object 
 8   p.symbol           75781 non-null  object 
 9   open_price         75781 non-null  float64
 10  high_price         75781 non-null  float64
 11  low_price          75781 non-null  float64
 12  close_price        75781 non-null  float64
 13  adj_close          75781 non-null  float64
 14  daily_volume       75781 non-null  int64  
 15  unadjusted_volume  75781 non-null  int64  
 16  change_dollars     757

In [11]:
cursor.close()
db.close()

### Cleaning Data

In [13]:
train_df = train_df.drop(['p.id', 'p.earnings_date', 'p.symbol', 't.id', 't.earnings_date', 't.symbol'], axis = 1)
print(train_df.head())

                id earnings_date symbol   eps  eps_estimated earnings_time  \
0   2012-01-09-AYI        1/9/12    AYI  0.74           0.67           bmo   
1   2012-01-09-HWM        1/9/12    HWM -0.09          -0.09           bmo   
2  2012-01-09-SCHN        1/9/12   SCHN  0.25           0.23           bmo   
3  2012-01-09-WDFC        1/9/12   WDFC  0.42           0.54           amc   
4   2012-01-10-SNX       1/10/12    SNX  1.37           1.14           amc   

   open_price  high_price  low_price  close_price  ...   sma_5  sma_10  \
0   54.980000   57.810001  52.250000    54.939999  ...  50.715  50.374   
1   20.824589   21.319340  20.734632    21.206898  ...   5.909   5.708   
2   42.770000   44.990002  42.750000    44.180000  ...  32.557  31.835   
3   40.290001   41.560001  39.840000    41.080002  ...  34.203  34.365   
4   15.895000   16.055000  15.800000    16.045000  ...  14.678  14.448   

   sma_20   ema_5  ema_10  ema_20  rsi_14   wma_5  wma_10  wma_20  
0  49.102  50.717 

In [17]:

is_NaN = train_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
forecast_these = train_df[row_has_NaN]
print(len(forecast_these))
print(forecast_these.head())

203
                    id earnings_date symbol  eps  eps_estimated earnings_time  \
75578   2022-01-18-BFC      01/18/22    BFC  NaN           1.39           bmo   
75579    2022-01-18-BK      01/18/22     BK  NaN           1.02           bmo   
75580  2022-01-18-CNXC      01/18/22   CNXC  NaN           2.54           amc   
75581  2022-01-18-CTXS      01/18/22   CTXS  NaN           0.43           bmo   
75582   2022-01-18-FBK      01/18/22    FBK  NaN           0.82           bmo   

       open_price  high_price  low_price  close_price  ...    sma_5   sma_10  \
75578       71.45      72.580      70.61        71.64  ...   72.376   72.432   
75579       63.20      63.690      62.16        63.60  ...   63.334   61.959   
75580      170.21     173.120     163.33       166.64  ...  169.310  172.466   
75581       94.53      96.260      94.45        95.80  ...   96.558   96.766   
75582       45.63      46.971      45.54        46.92  ...   46.406   45.870   

         sma_20       ema_5 

In [19]:
train_df = train_df[train_df["eps"].notna()]
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75578 entries, 0 to 75577
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 75578 non-null  object 
 1   earnings_date      75578 non-null  object 
 2   symbol             75578 non-null  object 
 3   eps                75578 non-null  float64
 4   eps_estimated      75578 non-null  float64
 5   earnings_time      75578 non-null  object 
 6   open_price         75578 non-null  float64
 7   high_price         75578 non-null  float64
 8   low_price          75578 non-null  float64
 9   close_price        75578 non-null  float64
 10  adj_close          75578 non-null  float64
 11  daily_volume       75578 non-null  int64  
 12  unadjusted_volume  75578 non-null  int64  
 13  change_dollars     75578 non-null  float64
 14  change_percent     75578 non-null  float64
 15  vwap               75578 non-null  float64
 16  change_over_time   755

### Feature Engineering